<a href="https://colab.research.google.com/github/Wajih24/AI-Workshop-Part1/blob/main/Notebook_AI_workshop_(Part1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integrating AI: Practical Development for Smart Applications

## 1.Setup

In [ ]:
%%capture
! pip install openai tiktoken transformers bertviz torch
! pip install --upgrade langchain           # core
! pip install --upgrade langchain-openai    # OpenAI integration
! pip install --upgrade langchain-google-genai  # Gemini integration
! pip install --upgrade langchain  langchain-openai  langchain-community duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13

## 2.Tokenization Demo & Attention Visualization

### 2.1 Tokenization Demo

In [ ]:
import tiktoken

# 1. Choose the encoding for your model (e.g. “gpt2” or “cl100k_base” for GPT-4)
enc = tiktoken.get_encoding("cl100k_base")

text = "The quick brown fox jumped over the lazy dog."
tokens = enc.encode(text)
token_strs = [enc.decode([t]) for t in tokens]

print("Original:", text)
print("Token IDs:", tokens)
print("Decoded tokens:", token_strs)

Original: The quick brown fox jumped over the lazy dog.
Token IDs: [791, 4062, 14198, 39935, 27096, 927, 279, 16053, 5679, 13]
Decoded tokens: ['The', ' quick', ' brown', ' fox', ' jumped', ' over', ' the', ' lazy', ' dog', '.']


### 2.2 Attention Visualization

In [ ]:
from transformers import GPT2Tokenizer, GPT2Model
from bertviz import head_view
import torch

# Load a small pretrained model
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2Model.from_pretrained(model_name, output_attentions=True)

# Prepare inputs
sentence = "The quick brown fox jumps over the lazy dog"
inputs = tokenizer(sentence, return_tensors="pt")

# Forward pass
outputs = model(**inputs)
attentions = outputs.attentions  # tuple: one Tensor per layer

# Visualize the attention of layer 0, head 0
# Must be run in a Jupyter environment
head_view(
    attentions,
    tokenizer.convert_ids_to_tokens(inputs["input_ids"][0]),
    layer=0
)


<IPython.core.display.Javascript object>

## 3. First Interaction

In [ ]:
# OpenAi
from openai import OpenAI
client = OpenAI(api_key = "...")

response = client.responses.create(
    model="gpt-4.1",
    input="Explain how AI works in a few words"
)

print(response.output_text)

AI works by using algorithms and data to recognize patterns, make decisions, and learn from experience—mimicking aspects of human intelligence.


In [ ]:
# Gemini
from google import genai

client = genai.Client(api_key="...")

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Explain how AI works in a few words",
)

print(response.text)

AI learns patterns from data to make predictions or decisions.



## 4. Zero-Shot vs. Few-Shot Prompting

### 4.1 Using OpenAI

In [ ]:
from openai import OpenAI

client = OpenAI(api_key = "...")
MODEL = "gpt-4o-mini"

def show(title: str, msgs: list[dict]):
    print("="*60, "\n", title)
    reply = client.chat.completions.create(
        model=MODEL,
        temperature=0,      # deterministic so we can compare
        messages=msgs
    )
    print(reply.choices[0].message.content.strip(), "\n")

# ------------------------------------------------------------------
# 1️⃣  ZERO-SHOT
# ------------------------------------------------------------------
zero_shot_messages = [
    {"role": "system", "content": (
        "You are a helpful assistant that tags customer e-mails as "
        "'positive', 'neutral', or 'negative'. Reply with only the tag."
    )},
    {"role": "user", "content": (
        "Hi team, thanks a lot for fixing the bug so quickly! "
        "The app works perfectly now. Cheers 😊"
    )}
]

show("ZERO-SHOT RESULT", zero_shot_messages)

# ------------------------------------------------------------------
# 2️⃣  FEW-SHOT
# ------------------------------------------------------------------
few_shot_messages = [
    {"role": "system", "content": (
        "You are a helpful assistant that tags customer e-mails as "
        "'positive', 'neutral', or 'negative'. Reply with only the tag."
    )},

    # ---- Few-shot examples ----
    {"role": "user",      "content": "I love the new interface 🎉"},
    {"role": "assistant", "content": "positive"},
    {"role": "user",      "content": "When is dark-mode coming?"},
    {"role": "assistant", "content": "neutral"},
    {"role": "user",      "content": "Your last update broke everything!"},
    {"role": "assistant", "content": "negative"},
    # ---------------------------

    # 🆕 Query to classify
    {"role": "user",      "content": (
        "Hi team, thanks a lot for fixing the bug so quickly! "
        "The app works perfectly now. Cheers 😊"
    )}
]

show("FEW-SHOT RESULT", few_shot_messages)

 ZERO-SHOT RESULT
positive 

 FEW-SHOT RESULT
positive 



### 4.2 Using Gemini

In [ ]:
import os
import google.generativeai as genai

genai.configure(api_key='...')
MODEL = "gemini-2.0-flash"

def show(title: str, prompt, **kwargs):
    print("="*60, "\n", title)
    response = genai.GenerativeModel(MODEL).generate_content(prompt, **kwargs)
    print(response.text.strip(), "\n")

# ------------------------------------------------------------------
# 1️⃣  ZERO-SHOT
# ------------------------------------------------------------------
zero_shot_prompt = (
    "Tag the following customer e-mail as **positive, neutral, or negative**. "
    "Reply with only the tag.\n\n"
    "E-mail: Hi team, thanks a lot for fixing the bug so quickly! "
    "The app works perfectly now. Cheers 😊"
)
show("ZERO-SHOT RESULT", zero_shot_prompt, generation_config={"temperature": 0})

# ------------------------------------------------------------------
# 2️⃣  FEW-SHOT
# ------------------------------------------------------------------
few_shot_prompt = (
    "You are an assistant that tags customer e-mails as **positive, neutral, or negative**. "
    "Reply with only the tag.\n\n"
    # ---- Few-shot examples ----
    "E-mail: I love the new interface 🎉\n"
    "Tag: positive\n\n"
    "E-mail: When is dark-mode coming?\n"
    "Tag: neutral\n\n"
    "E-mail: Your last update broke everything!\n"
    "Tag: negative\n\n"
    # ---------------------------
    # 🆕  E-mail to classify
    "E-mail: Hi team, thanks a lot for fixing the bug so quickly! "
    "The app works perfectly now. Cheers 😊\n"
    "Tag:"
)
show("FEW-SHOT RESULT", few_shot_prompt, generation_config={"temperature": 0})

 ZERO-SHOT RESULT
Positive 

 FEW-SHOT RESULT
positive 



## 5. Chain-of-Thought

### 5.1 Basic Approach

In [ ]:
client = OpenAI(api_key = "...")
MODEL = "o4-mini"
# ----------   Build the prompt   ----------
messages = [
    {
        "role": "system",
        "content": (
            "You are a careful math tutor. "
            "When you solve a problem, first think **step by step**, "
            "then write your final answer on the last line beginning with "
            "'Final answer:'"
        ),
    },
    {
        "role": "user",
        "content": (
            "A bookstore sold 12 novels and twice as many comics. "
            "A novel costs $15, a comic costs $6. "
            "How much total revenue did the bookstore earn?"
        ),
    },
]

# ----------   Call the API   ----------
reply = client.chat.completions.create(
    model=MODEL,
    #temperature=0,          # deterministic
    messages=messages,
)

full_text = reply.choices[0].message.content.strip()
print("=== MODEL OUTPUT ===\n")
print(full_text)

# ----------   (Optional) extract the numeric answer   ----------
for line in full_text.splitlines():
    if line.lower().startswith("final answer"):
        print("\n→", line.split(":", 1)[1].strip())
        break

=== MODEL OUTPUT ===

Let’s work through the problem step by step:

1. Number of novels sold = 12  
   Price per novel = \$15  
   Revenue from novels = 12 × \$15 = \$180  

2. Number of comics sold = twice as many as novels = 2 × 12 = 24  
   Price per comic = \$6  
   Revenue from comics = 24 × \$6 = \$144  

3. Total revenue = Revenue from novels + Revenue from comics  
   Total revenue = \$180 + \$144 = \$324  

Final answer: \$324

→ \$324


In [ ]:
genai.configure(api_key='...')
MODEL = "gemini-2.0-flash"

# ----------   Build the prompt   ----------
prompt = (
    "You are a careful math tutor.\n"
    "When you solve a problem, first think **step by step**, "
    "then write your final answer on the last line beginning with "
    "'Final answer:'.\n\n"
    # 🆕  Problem
    "A bookstore sold 12 novels and twice as many comics. "
    "A novel costs $15, a comic costs $6. "
    "How much total revenue did the bookstore earn?"
)

# ----------   Call the API   ----------
response = genai.GenerativeModel(MODEL).generate_content(
    prompt,
    generation_config={"temperature": 0}   # deterministic reasoning
)

full_text = response.text.strip()
print("=== GEMINI OUTPUT ===\n")
print(full_text)

# ----------   (Optional) extract the numeric answer   ----------
for line in full_text.splitlines():
    if line.lower().startswith("final answer"):
        print("\n→", line.split(":", 1)[1].strip())
        break

=== GEMINI OUTPUT ===

Let's break this problem down step by step.

1. **Find the number of comics sold:** The bookstore sold twice as many comics as novels, and they sold 12 novels. So, the number of comics sold is 2 * 12 = 24 comics.

2. **Calculate the revenue from novels:** The bookstore sold 12 novels at $15 each. So, the revenue from novels is 12 * $15 = $180.

3. **Calculate the revenue from comics:** The bookstore sold 24 comics at $6 each. So, the revenue from comics is 24 * $6 = $144.

4. **Calculate the total revenue:** The total revenue is the sum of the revenue from novels and the revenue from comics. So, the total revenue is $180 + $144 = $324.

Final answer: $324

→ $324


### 5.2 LangChain Introduction & Integration

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


OPENAI_API_KEY = "..."
GEMINI_API_KEY = '...'

# --- 🔀  PICK YOUR MODEL  -------------------------------------------------
USE_OPENAI = True          # set False to switch to Gemini

if USE_OPENAI:
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key  = OPENAI_API_KEY)
else:
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash"  , temperature=0,  api_key  = GEMINI_API_KEY)
# -------------------------------------------------------------------------

# ---- Prompt with CoT instruction ---------------------------------------
template = """
You are a careful math tutor.
First, think **step by step**, then give your result on the last line
starting with 'Final answer:'.

Problem: {question}
"""

prompt = PromptTemplate(template=template, input_variables=["question"])
chain  = LLMChain(llm=llm, prompt=prompt)

# ---- Run ----------------------------------------------------------------
question = ("A bookstore sold 12 novels and twice as many comics. "
            "A novel costs $15, a comic costs $6. "
            "How much total revenue did the bookstore earn?")

result = chain.invoke({"question": question})
print("=== CHAIN OUTPUT ===\n")
print(result["text"])

# ---- Pull out the numeric answer ---------------------------------------
for line in result["text"].splitlines():
    if line.lower().startswith("final answer"):
        print("\n→", line.split(":", 1)[1].strip())
        break

<ipython-input-33-80caf96551e9>:29: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain  = LLMChain(llm=llm, prompt=prompt)


=== CHAIN OUTPUT ===

Let's break down the problem step by step.

1. **Determine the number of comics sold**: 
   The bookstore sold 12 novels and twice as many comics. 
   So, the number of comics sold is:
   \[
   2 \times 12 = 24 \text{ comics}
   \]

2. **Calculate the revenue from novels**: 
   Each novel costs $15. Therefore, the total revenue from novels is:
   \[
   12 \text{ novels} \times 15 \text{ dollars/novel} = 180 \text{ dollars}
   \]

3. **Calculate the revenue from comics**: 
   Each comic costs $6. Therefore, the total revenue from comics is:
   \[
   24 \text{ comics} \times 6 \text{ dollars/comic} = 144 \text{ dollars}
   \]

4. **Calculate the total revenue**: 
   Now, we add the revenue from novels and comics to find the total revenue:
   \[
   180 \text{ dollars} + 144 \text{ dollars} = 324 \text{ dollars}
   \]

Final answer: 324 dollars

→ 324 dollars


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import AgentExecutor, create_openai_tools_agent

OPENAI_API_KEY = "..."
GEMINI_API_KEY = '...'


USE_OPENAI = True          # set False to switch to Gemini

if USE_OPENAI:
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key  = OPENAI_API_KEY)
else:
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash"  , temperature=0,  api_key  = GEMINI_API_KEY)


# ─── 2. DEFINE TOOLS ─────────────────────────────────────────────────────
@tool
def calculator(expr: str) -> str:
    """Evaluate a simple space-separated arithmetic expression, e.g. '2 * 3 + 4'."""
    import operator as op
    ops = {"+": op.add, "-": op.sub, "*": op.mul, "/": op.truediv}
    stack = []
    for token in expr.split():
        if token in ops:
            b, a = stack.pop(), stack.pop()
            stack.append(ops[token](a, b))
        else:
            stack.append(float(token))
    return str(stack[0])

search = DuckDuckGoSearchRun()          # already a Tool instance
tools  = [calculator, search]

# ─── 3. BUILD THE PROMPT ─────────────────────────────────────────────────
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a helpful research assistant. "
         "Think step by step and use the provided tools when needed."),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),   # ← REQUIRED!
    ]
)

# ─── 4. CREATE THE AGENT & EXECUTOR ──────────────────────────────────────
agent_runnable = create_openai_tools_agent(llm, tools, prompt)   # ← fixed
agent          = AgentExecutor(agent=agent_runnable, tools=tools, verbose=True)

# ─── 5. RUN A QUERY ──────────────────────────────────────────────────────
question = (
    "Galileo was born in Pisa and died in Arcetri. "
    "How old was he when he died?"
)

result = agent.invoke({"input": question})
print("\n=== FINAL ANSWER ===\n", result["output"])



> Entering new AgentExecutor chain...
Galileo Galilei was born on February 15, 1564, and died on January 8, 1642. To calculate his age at the time of his death, we can subtract his birth year from his death year and then adjust for whether he had already had his birthday that year.

1. Death year: 1642
2. Birth year: 1564
3. Age at death: 1642 - 1564 = 78 years

Since he died before his birthday in 1642, he was 77 years old when he died.

> Finished chain.

=== FINAL ANSWER ===
 Galileo Galilei was born on February 15, 1564, and died on January 8, 1642. To calculate his age at the time of his death, we can subtract his birth year from his death year and then adjust for whether he had already had his birthday that year.

1. Death year: 1642
2. Birth year: 1564
3. Age at death: 1642 - 1564 = 78 years

Since he died before his birthday in 1642, he was 77 years old when he died.


In [ ]:
import uuid
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools   import tool
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history      import InMemoryChatMessageHistory
from langchain_community.tools        import DuckDuckGoSearchRun
from langchain.agents import AgentExecutor, create_openai_tools_agent

# ── 1. Choose model ──────────────────────────────────────────────────────
OPENAI_API_KEY = "..."
GEMINI_API_KEY = '...'


USE_OPENAI = True          # set False to switch to Gemini

if USE_OPENAI:
    from langchain_openai import ChatOpenAI
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key  = OPENAI_API_KEY)
else:
    from langchain_google_genai import ChatGoogleGenerativeAI
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash"  , temperature=0,  api_key  = GEMINI_API_KEY)

# ── 2. Tools ─────────────────────────────────────────────────────────────
@tool
def calculator(expr: str) -> str:
    """Evaluate a space-separated arithmetic expression (e.g. '2 * 3 + 4')."""
    import operator as op
    ops, st = {"+": op.add, "-": op.sub, "*": op.mul, "/": op.truediv}, []
    for tok in expr.split():
        st.append(float(tok) if tok not in ops else ops[tok](st.pop(-2), st.pop()))
    return str(st[0])

search = DuckDuckGoSearchRun()
tools  = [search]

# ── 3. Prompt with history & scratchpad placeholders ─────────────────────
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are a helpful research assistant. "
         "Think step by step and use the provided tools when helpful."),
        MessagesPlaceholder("history"),          # ← past turns
        ("human", "{input}"),                    # ← user’s new message
        MessagesPlaceholder("agent_scratchpad"), # ← tool calls / thoughts
    ]
)

# ── 4. Build single-turn agent (no memory yet) ──────────────────────────
agent_core   = create_openai_tools_agent(llm, tools, prompt)
agent_single = AgentExecutor(agent=agent_core, tools=tools, return_messages=True, verbose=True)

# ── 5. Wrap with chat-history memory ─────────────────────────────────────
def new_history(_session_id: str) -> InMemoryChatMessageHistory:  # can swap for Redis, PG…
    return InMemoryChatMessageHistory()

agent_chat = RunnableWithMessageHistory(
    agent_single,
    new_history,
    input_messages_key="input",
    history_messages_key="history",
    output_messages_key="output",
)

# ── 6. Tiny CLI loop ─────────────────────────────────────────────────────
session_id = str(uuid.uuid4())   # one chat session
print("🤖  Conversation agent ready!  (empty line to exit)\n")
while True:
    try:
        user = input("🧑 You: ").strip()
    except (KeyboardInterrupt, EOFError):
        break
    if not user:
        break

    result = agent_chat.invoke(
        {"input": user},
        config={"configurable": {"session_id": session_id}},
    )
    print("🤖 Bot:", result["output"], "\n")

🤖  Conversation agent ready!  (empty line to exit)

🧑 You: 5
🤖 Bot: It seems like your message is incomplete. Could you please provide more details or clarify what you need assistance with? 

🧑 You: 
